In [71]:
#Importing Necessary Libraries.
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./cell_images"))

# Any results you write to the current directory are saved as output.

['Parasitized', 'Uninfected']


In [74]:
data=[]
labels=[]
Parasitized=os.listdir("./cell_images/Parasitized/")
for a in Parasitized:
    try:
        image=cv2.imread("./cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

Uninfected=os.listdir("./cell_images/Uninfected/")
for b in Uninfected:
    try:
        image=cv2.imread("./cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")
    

In [77]:
Cells=np.array(data)
labels=np.array(labels)

np.save("Cells",Cells)
np.save("labels",labels)

Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

In [78]:
s=np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

num_classes=len(np.unique(labels))
len_data=len(Cells)

(x_train,x_test)=Cells[(int)(0.1*len_data):],Cells[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

In [79]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

print(y_test)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [13]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()

W0623 05:43:09.783881 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 05:43:09.893016 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 05:43:09.924344 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 05:43:09.974425 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0623 05:43:10.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
__________

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=50,epochs=20,verbose=1)

W0623 05:43:17.230395 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0623 05:43:17.340785 140660139689792 deprecation_wrapper.py:119] From /home/andrefu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0623 05:43:17.538551 140660139689792 deprecation.py:323] From /home/andrefu/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
24803/24803 [==============================] - 34s 1ms/step - loss: 0.3619 - acc: 0.8346
Epoch 2/20
24803/24803 [==============================] - 26s 1ms/step - loss: 0.1671 - acc: 0.9457
Epoch 3/20
24803/24803 [==============================] - 26s 1ms/step - loss: 0.1411 - acc: 0.9558
Epoch 4/20
24803/24803 [==============================] - 29s 1ms/step - loss: 0.1270 - acc: 0.9588
Epoch 5/20
24803/24803 [==============================] - 27s 1ms/step - loss: 0.1174 - acc: 0.9610
Epoch 6/20
24803/24803 [==============================] - 26s 1ms/step - loss: 0.1085 - acc: 0.9617
Epoch 7/20
24803/24803 [==============================] - 27s 1ms/step - loss: 0.1023 - acc: 0.9647
Epoch 8/20
24803/24803 [==============================] - 26s 1ms/step - loss: 0.0965 - acc: 0.9668
Epoch 9/20
24803/24803 [==============================] - 27s 1ms/step - loss: 0.0857 - acc: 0.9703
Epoch 10/20
24803/24803 [==============================] - 27s 1ms/step - loss: 0.0940 - acc: 0.9672

In [30]:
from sklearn.metrics import classification_report , accuracy_score
classes = model.predict_classes(x_test)
print(classes)
print(np.argmax(y_test, axis=1))
print(classification_report(np.argmax(y_test, axis=1), classes))

accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test_Accuracy:-', accuracy[1])

[1 1 1 ... 0 0 1]
[1 1 1 ... 0 0 1]
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1362
           1       0.96      0.96      0.96      1393

    accuracy                           0.96      2755
   macro avg       0.96      0.96      0.96      2755
weighted avg       0.96      0.96      0.96      2755

2755/2755 [==============================] - 1s 261us/step

 Test_Accuracy:- 0.9575317604355716


In [28]:
from keras.models import load_model
model.save('cells.h5')

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import argparse
import cv2
import os

In [35]:
def quantify_image(image):
	# compute the histogram of oriented gradients feature vector for
	# the input image
	features = feature.hog(image, orientations=9,
		pixels_per_cell=(10, 10), cells_per_block=(2, 2),
		transform_sqrt=True, block_norm="L1")
 
	# return the feature vector
	return features

In [63]:
print(os.listdir('./spiral/training'))

['healthy', 'parkinson']


In [42]:
data=[]
labels=[]
healthy=os.listdir("./spiral/training/healthy/")
for a in healthy:
    try:
        image=cv2.imread("./spiral/training/healthy/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

park=os.listdir("./spiral/training/parkinson/")
for b in Uninfected:
    try:
        image=cv2.imread("./spiral/training/parkinson/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")
        
healthy = os.listdir("./spiral/testing/healthy/")
for a in healthy:
    try:
        image=cv2.imread("./spiral/testing/healthy/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

park=os.listdir("./spiral/testing/parkinson/")
for b in Uninfected:
    try:
        image=cv2.imread("./spiral/testing/parkinson/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")
    

In [64]:
spiral=np.array(data)
labels=np.array(labels)

np.save("spiral",spiral)
np.save("labels_s",labels)

spiral=np.load("spiral.npy")
labels=np.load("labels_s.npy")

In [91]:
s=np.arange(spiral.shape[0])
np.random.shuffle(s)
spiral=spiral[s]
labels=labels[s]

num_classes=len(np.unique(labels))
len_data=len(spiral)

(x_train,x_test)=spiral[(int)(0.1*len_data):],spiral[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

In [92]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

print(y_test)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [93]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 6, 6, 64)          0         
__________

In [95]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=30,epochs=50,verbose=1)

Epoch 1/50
46/46 [==============================] - 1s 16ms/step - loss: 0.6766 - acc: 0.6304
Epoch 2/50
46/46 [==============================] - 0s 1ms/step - loss: 0.6833 - acc: 0.5435
Epoch 3/50
46/46 [==============================] - 0s 1ms/step - loss: 0.6708 - acc: 0.6739
Epoch 4/50
46/46 [==============================] - 0s 2ms/step - loss: 0.7213 - acc: 0.4348
Epoch 5/50
46/46 [==============================] - 0s 2ms/step - loss: 0.7035 - acc: 0.4783
Epoch 6/50
46/46 [==============================] - 0s 2ms/step - loss: 0.6707 - acc: 0.6304
Epoch 7/50
46/46 [==============================] - 0s 2ms/step - loss: 0.6775 - acc: 0.6087
Epoch 8/50
46/46 [==============================] - 0s 2ms/step - loss: 0.6833 - acc: 0.5217
Epoch 9/50
46/46 [==============================] - 0s 2ms/step - loss: 0.6936 - acc: 0.5217
Epoch 10/50
46/46 [==============================] - 0s 2ms/step - loss: 0.6797 - acc: 0.5217
Epoch 11/50
46/46 [==============================] - 0s 1ms/step - l

In [ ]:
from sklearn.metrics import classification_report , accuracy_score
classes = model.predict_classes(x_test)
print(classes)
print(np.argmax(y_test, axis=1))
print(classification_report(np.argmax(y_test, axis=1), classes))

accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test_Accuracy:-', accuracy[1])

In [59]:
spiral=np.load("spiral.npy")
labels=np.load("labels_s.npy")

In [70]:
print(y_test)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
